# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [69]:
TRAINING_DATA_FILE = "SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE = "SpamData/03_Testing/prob-nonspam.txt"
TOKEN_ALL_PROB_FILE = "SpamData/03_Testing/prob-all-tokens.txt"

VOCAB_SIZE = 2500

# Read and load features from .txt to numpy array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter = " ", dtype = int)
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter = " ", dtype = int)

In [4]:
print("Number of rows of training data", sparse_train_data.shape[0])
print("Number of rows of testing data", sparse_test_data.shape[0])
print("Number of emails for training", np.unique(sparse_train_data[:, 0]).size)
print("Number of emails for testing", np.unique(sparse_test_data[:, 0]).size)
type(sparse_train_data)

Number of rows of training data 258346
Number of rows of testing data 117777
Number of emails for training 4013
Number of emails for testing 1724


numpy.ndarray

### Create an empty dataframe

In [5]:
column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(0, VOCAB_SIZE))
len(column_names)

2502

In [6]:
index_names = np.unique(sparse_train_data[:, 0])
len(index_names)

4013

In [7]:
full_train_data = pd.DataFrame(index = index_names, columns = column_names)
full_train_data.fillna(value = 0, inplace = True)
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Convert sparse matrix to full matrix

In [8]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx = 0, word_idx = 1, cat_idx = 2, freq_idx = 3):
    """
    Form a full matrix from a sparse matrix.
    Return a pandas dataframe.
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column.
    word_idx -- poosition of the word id in the sparse matrix. Default: 2rd column.
    cat_idx -- position of the label (spam is 1, nonspam is 0). Default: 3rd column.
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column.
    """
    column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value = 0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, "DOC_ID"] = doc_nr
        full_matrix.at[doc_nr, "CATEGORY"] = label
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index("DOC_ID", inplace = True)
    return full_matrix
    

In [9]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 8.04 s


In [10]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0


# Training the Naive Bayes Model

## Calculating the Probability of Spam

In [21]:
# Probability of Spam
prob_spam = len(full_train_data[full_train_data['CATEGORY'] == 1]) / full_train_data.shape[0]
print("The probability of of spam is:",prob_spam)

The probability of of spam is: 0.310989284824321


## Total Number of Words / Tokens

In [22]:
full_train_features = full_train_data.loc[:, full_train_data.columns != "CATEGORY"]
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0


In [25]:
email_lengths = full_train_features.sum(axis = 1)
print(email_length.shape)
email_lengths[: 5]

(4013,)


DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [27]:
# Total word count
total_wc = email_lengths.sum()
print(total_wc)

429217


## Number of Tokens in Spam & Ham Emails

In [39]:
#spam_lengths = full_train_data[full_train_data['CATEGORY'] == 1].loc[:, full_train_data.columns != "CATEGORY"].sum(axis = 1)
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
#ham_lengths = full_train_data[full_train_data['CATEGORY'] == 0].loc[:, full_train_data.columns != "CATEGORY"].sum(axis = 1)
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
spam_wc = spam_lengths.sum()
ham_wc = ham_lengths.sum()
print(spam_lengths, ham_lengths)
print(spam_wc, ham_wc)

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
1885     45
1887     26
1889    115
1890     26
1895    162
Length: 1248, dtype: int64 DOC_ID
1896     82
1898    112
1899     66
1900     49
1901     34
       ... 
5789     18
5790     72
5791     46
5794     57
5795    264
Length: 2765, dtype: int64
176327 252890


In [40]:
spam_wc + ham_wc - total_wc

0

In [45]:
print("Average number of words in spam emails is {:.0f}".format(spam_wc / spam_lengths.shape[0]))
print("Average number of words in ham emails is {:.3f}".format(ham_wc / ham_lengths.shape[0]))

Average number of words in spam emails is 141
Average number of words in ham emails is 91.461


## Summing the Tokens Occuring in Spam and Ham

In [46]:
full_train_features.shape

(4013, 2500)

In [48]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]

In [59]:
# Laplace Smoothing (plus the extra one)
summed_spam_tokens = train_spam_tokens.sum(axis = 0) + 1
summed_ham_tokens = train_ham_tokens.sum(axis = 0) + 1
print(summed_spam_tokens)
print(summed_ham_tokens)

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495      22
2496       2
2497       1
2498       6
2499      21
Length: 2500, dtype: int64
0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495       7
2496       1
2497      33
2498      25
2499      10
Length: 2500, dtype: int64


## P(Token | Spam) - Probability that a token occurs if the email is a spam. 

In [64]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
print(prob_tokens_spam)
print(prob_tokens_spam.sum())

0       0.012185
1       0.005229
2       0.006805
3       0.011307
4       0.006817
          ...   
2495    0.000123
2496    0.000011
2497    0.000006
2498    0.000034
2499    0.000117
Length: 2500, dtype: float64
1.0


## P(Token | Ham) - Probability that a token occurs if the email is legit. 

In [65]:
prob_tokens_ham = summed_ham_tokens / (ham_wc + VOCAB_SIZE)
print(prob_tokens_ham)
print(prob_tokens_ham.sum())

0       0.021473
1       0.010141
2       0.008007
3       0.003673
4       0.006312
          ...   
2495    0.000027
2496    0.000004
2497    0.000129
2498    0.000098
2499    0.000039
Length: 2500, dtype: float64
1.0


## P(Token) - Probability that a token occurs.

In [66]:
prob_tokens_all = full_train_features.sum(axis = 0) / total_wc
print(prob_tokens_all)
print(prob_tokens_all.sum())

0       0.017849
1       0.008208
2       0.007595
3       0.006892
4       0.006591
          ...   
2495    0.000063
2496    0.000002
2497    0.000075
2498    0.000068
2499    0.000068
Length: 2500, dtype: float64
1.0


# Save the statistics.

In [71]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare testing data

In [158]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 3.44 s


In [159]:
X_test = full_test_data.loc[:, full_test_data.columns != "CATEGORY"]
#y_test = full_test_data.loc[:, full_test_data.columns == "CATEGORY"]
y_test = full_test_data.CATEGORY
np.savetxt(TEST_FEATURE_MATRIX, X_test)
np.savetxt(TEST_TARGET_FILE, y_test)